# 02 – Preprocesamiento de datos de criptomonedas

Este notebook realiza el preprocesamiento de los datos descargados en:

`01_descarga_datos.ipynb`

Tareas ejecutadas:

1. Cargar los datos crudos en `../data/raw/criptomonedas_2018_2020_raw.csv`
2. Verificar calidad y consistencia
3. Calcular retornos logarítmicos diarios
4. Guardar los resultados en `../data/processed/` para el pipeline TDA.

Este notebook forma parte de la investigación:
_“Análisis topológico de criptomonedas mediante paisajes de persistencia (SEIO 2023)”_.

In [ ]:
# ===============================================
# 1. Importación de librerías
# ===============================================

import pandas as pd
from pathlib import Path
import numpy as np

print("Librerías cargadas correctamente.")

## 2. Cargar datos crudos desde `data/raw/`

In [ ]:
raw_file = Path("../data/raw/criptomonedas_2018_2020_raw.csv")

if not raw_file.exists():
    raise FileNotFoundError(f"No se encuentra el archivo: {raw_file}")

prices_df = pd.read_csv(raw_file, index_col=0, parse_dates=True)
prices_df = prices_df.sort_index()

print("Dimensiones del DataFrame cargado:", prices_df.shape)
prices_df.head()

## 3. Verificación de valores faltantes y consistencia

Se revisan valores perdidos y rangos anómalos. Si existen fechas sin información se eliminan, ya que el TDA requiere vectores completos por cada día.

In [ ]:
# Valores faltantes
null_counts = prices_df.isnull().sum()
print("Valores nulos por columna:\n", null_counts)

# Eliminación de filas incompletas
prices_df = prices_df.dropna()
print("\nTras eliminar NA, dimensiones:", prices_df.shape)

## 4. Cálculo de retornos logarítmicos diarios

La transformación aplicada es:

$$ r_t = \log(p_t) - \log(p_{t-1}) $$

Se eliminan las primeras observaciones no computables.

In [ ]:
log_returns = np.log(prices_df / prices_df.shift(1)).dropna()
print("Dimensiones de retornos logarítmicos:", log_returns.shape)
log_returns.head()

## 5. Guardar datos procesados en `data/processed/`

In [ ]:
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

file_prices = processed_dir / "criptos_precios_2018_2020.csv"
file_returns = processed_dir / "criptos_retornos_log_2018_2020.csv"

prices_df.to_csv(file_prices)
log_returns.to_csv(file_returns)

print("Archivos procesados guardados:")
print(" -", file_prices)
print(" -", file_returns)

## 6. Resumen estadístico (opcional)

In [ ]:
summary = log_returns.describe().T
summary